# Contents

TOC stopped working for some reason...

- Users
  - User States
  - Disabled Users
- [Privileged administration](#privileged-administration)
  - [Limit the number of Global Administrators to less than 5 (*high*)](#limit-the-number-of-global-administrators-to-less-than-5)
  - [Synchronized accounts (*high*)](#synchronized-accounts)
  - [Use groups for Azure AD role assignments (*high*)](#use-groups-for-azure-ad-role-assignments)
  - [PIM Alerts (*high*)](#pim-alerts)
  - [Recurring access reviews (*high*)](#recurring-access-reviews)
  - [Access Reviews: Enabled for all groups (*high*)](#access-reviews:-enabled-for-all-groups)
  - [Apps and Owners Can Change All Group Membership](#apps-and-owners-can-change-all-group-membership)
  - [Avoid standing access for user accounts and permissions](#avoid-standing-access-for-user-accounts-and-permissions)
- [External Identities](#external-identities)
  - [Guest Invite Settings (*high*)](#guest-invite-settings)
  - [Guest User Access Restrictions (*high*)](#guest-user-access-restrictions)
- [User Setting](#user-setting)
  - [User role permissions (Application registration) (*high*)](#user-role-permissions-application-registration)
  - [Authentication Methods](#authentication-methods)
- [Custom Domains](#custom-domains)
  - [Verified Domains (*high*)](#verified-domains)
- [Enterprise Applications](#enterprise-applications)
  - [User Consent for Apps (*medium*)](#user-consent-for-apps)
  - [Group Owner Consent (*medium*)](#group-owner-consent)
  - [Application Owners (*high*)](#application-owners)
- [Conditional Access Policies](#conditional-access-policies)
  - [Block Legacy Protocols (*high*)](#block-legacy-protocols)
  - [Require MFA for Administrators (*high*)](#require-mfa-for-administrators)
  - [Require MFA for Azure Management (*high*)](#require-mfa-for-azure-management)
  - [Restricted Locations (*medium*)](#restricted-locations)
  - [Require devices to be marked as compliant (*high*)](#require-devices-to-be-marked-as-compliant)
- [Password Reset](#password-reset)
  - [Self-service password reset policy requirement](#self-service-password-reset-policy-requirement)
  - [Re-confirm authentication information](#re-confirm-authentication-information)
  - [Number of methods required to reset password](#number-of-methods-required-to-reset-password)

# Initialization

In [1]:
# Run this to reset
$TenantId = $null
$Token = $null

In [ ]:
# Set this to true to also output, where applicable, to Markdown
$OutputMarkdown = $true

$TenantId = if($null -eq $TenantId){ Read-Host -Prompt "Enter tenant ID" } else { $TenantId }

# connect once for all necessary scopes for this notebook - these are delegated permissions so we cannot do something the authenticated user could not already do!
# This means that 'Microsoft Graph Command Line Tools' must be approved by a Global Administrator
# Disconnect-Graph
# TODO: use Get-MgContext to check if we need to connect again
# NOTE: Never request a write scope!
$Scopes = "Directory.AccessAsUser.All", "Policy.Read.All", "RoleManagement.Read.Directory", "RoleManagementAlert.Read.Directory", "AccessReview.Read.All", "Application.Read.All", "Directory.Read.All", "AuditLog.Read.All", "CrossTenantInformation.ReadBasic.All"
$null = Connect-MgGraph -Scopes $Scopes -TenantId $TenantId -ContextScope Process -ErrorAction Stop -NoWelcome

Write-Host "Connected to tenant '$TenantId' with the following scope: $Scopes"
$null = Set-AzContext -TenantId $TenantId -ErrorAction Stop

# load functions
. "..\src\functions.ps1"

<#  https://microsoft.com/devicelogin  #>
# $Token = if ([string]::IsNullOrEmpty($Token)) { Get-DeviceCodeAuthenticationToken -tenantId $TenantId -Verbose } else { $Token }

In [ ]:
# If you are having trouble with continous prompting try to run this (remove comments)

# Disconnect-MgGraph
# Connect-MgGraph

# Users

## User states

Count the total number of users, disabled users, deleted users, and guest users.

In [ ]:
Get-UserStates -OutputToHost

## Disabled Users

Find disabled users with group memberships or roles or licenses assigned.

Disabled users should not have roles or licenses assigned, and group memberships should at least be reviewed. 

In [ ]:
Get-DisabledUsers -IncludeLicenseDetails -OutputMarkdown:$OutputMarkdown

In [ ]:
Get-DisabledUsers -IncludeGroupMemberships -OutputMarkdown:$OutputMarkdown

# Privileged administration

## Limit the number of Global Administrators to less than 5

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

[Entra ID best practice](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#5-limit-the-number-of-global-administrators-to-less-than-5)

*As a best practice, Microsoft recommends that you assign the Global Administrator role to fewer than five people in your organization...*

In [ ]:
Get-GlobalAdminstrators -OutputMarkdown:$OutputMarkdown

## Synchronized accounts

*Severity*: High

*Guid*: 87791be1-1eb0-48ed-8003-ad9bcf241b99

Do not synchronize accounts with the highest privilege access to on-premises resources as you synchronize your enterprise identity systems with cloud directories.

If below list any users then `onPremisesSyncEnabled` is true (and their account is enabled). Those should have the role removed, and a cloud-only user created as a replacement.

[Entra ID best practice](https://learn.microsoft.com/en-us/azure/security/fundamentals/identity-management-best-practices#centralize-identity-management)

*Don’t synchronize accounts to Azure AD that have high privileges in your existing Active Directory instance...*

In [ ]:
Get-SynchronizedAccounts -OutputMarkdown:$OutputMarkdown

## Use groups for Entra ID role assignments

*Work in Progress*

For now we can check the *Membership* column in [Privileged Identity Management | Azure AD roles](https://portal.azure.com/?feature.msaljs=true#view/Microsoft_Azure_PIMCommon/ResourceMenuBlade/~/members/resourceId//resourceType/tenant/provider/aadroles)

*Severity*: High

*Guid*: e0d968d3-87f6-41fb-a4f9-d852f1673f4c

[Best Practice: Use groups for Microsoft Entra role assignments and delegate the role assignment](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#7-use-groups-for-microsoft-entra-role-assignments-and-delegate-the-role-assignment)

*If you have an external governance system that takes advantage of groups, then you should consider assigning roles to Microsoft Entra groups, instead of individual users....*


In [ ]:
Get-GroupsWithRoleAssignments -OutputMarkdown:$OutputMarkdown

## PIM Alerts

*Severity*: High

*Guid*: N/A

There should be no active alerts in PIM. If below identifies any active alerts go to [PIM alerts](https://portal.azure.com/#view/Microsoft_Azure_PIMCommon/ResourceMenuBlade/~/Alerts/resourceId//resourceType/tenant/provider/aadroles) for further details.

In [ ]:
Get-PimAlerts -OutputMarkdown:$OutputMarkdown

We can also list affected principals. Note that in some cases there is no direct principal, ex. for the alert `NoMfaOnRoleActivationAlert`

In [ ]:
Get-PimAlertAffectedPrincipals -OutputMarkdown:$OutputMarkdown

## Recurring access reviews

*Severity*: High

*Guid*: eae64d01-0d3a-4ae1-a89d-cc1c2ad3888f

Configure recurring access reviews to revoke unneeded permissions over time.

[Best Practice: Configure recurring access reviews to revoke unneeded permissions over time](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#4-configure-recurring-access-reviews-to-revoke-unneeded-permissions-over-time)

If there are no access review definitions then there are no recurring access reviews.

In [ ]:
Get-RecurringAccessReviews

## Access Reviews: Enabled for all groups

*Severity*: Medium

*Guid*: e6b4bed3-d5f3-4547-a134-7dc56028a71f

[Plan a Microsoft Entra access reviews deployment](https://learn.microsoft.com/en-us/azure/active-directory/governance/deploy-access-reviews)

## Apps and Owners Can Change All Group Membership

*Work in Progress*

Chad Cox: Group Membership changes to all groups, this script list every role and member (not pim eligible) with this capability , every application with the permission, and every owner of the application. Some of the permissions are to unified and some are to security. either way can you imagine if someone granted access to a group that gave them all kinds of access to teams sites or access to other cloud resources.

https://www.linkedin.com/posts/chad-cox-194bb560_entraid-aad-azuread-activity-7093368251329495040-Ff9T

https://github.com/chadmcox/Azure_Active_Directory/blob/master/Applications/get-AppsandOwnersCanChangeAllGroupMembership.ps1

TODO: does not show the display name for Owners of relevant apps

In [ ]:
Test-AppOwnersChangeGroupMembership -OutputMarkdown:$OutputMarkdown

## Avoid standing access for user accounts and permissions

*Work in Progress*

[MCSB: PA-2: Avoid standing access for user accounts and permissions](https://learn.microsoft.com/en-us/security/benchmark/azure/mcsb-privileged-access#pa-2-avoid-standing-access-for-user-accounts-and-permissions)

In [ ]:
Test-StandingAccess -OutputMarkdown:$OutputMarkdown -ShowProgress

# External Identities

## Guest invite settings

*Severity*: High

*Guid*: be64dd7d-f2e8-4bbb-a468-155abc9164e9

External Collaboration Settings: Guest invite settings set to `'Only users assigned to specific admin roles can invite guest users'` or `'No one in the organization can invite guest users including admins (most restrictive)'`

In [ ]:
Test-GuestInviteSettings -OutputToHost

## Guest user access restrictions

*Work in Progress*

*Severity*: High

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

External Collaboration Settings: Guest user access set to `'Guest user access is restricted to properties and memberships of their own directory objects (most restrictive)'`

In [ ]:
Test-GuestUserAccessRestrictions

# User Setting

## User role permissions (Application registration)

*Severity*: High

*Guid*: a2cf2149-d013-4a92-9ce5-74dccbd8ac2a

Users can register applications should be set to `No`.

Users should not be allowed to register applications. Use specific roles such as `Application Developer`.

In [ ]:
Test-UsersCanRegisterApplications -OutputToHost -ShowExplanation

## Authentication Methods

*Work in Progress*

Check if authentication method policies are enabled or not

check if migration has already been done, and if not, can we check if the methods in the different places are enabled in the policy

In [ ]:
Test-AuthenticationMethods

# Custom Domains

## Verified Domains

*Severity*: High

*Guid*: bade4aad-1e8c-439e-a946-667313c00567

Only validated customer domains are registered

In [ ]:
Test-VerifiedDomains -OutputToHost -ShowExplanation

# Enterprise Applications

##  User consent for apps

*Severity*: Medium

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

Consent & Permissions: Allow user consent for apps from verified publishers

[Configure how users consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent?pivots=ms-graph)

In [ ]:
Test-UserConsentForApps -OutputToHost -ShowExplanation

##  Group Owner Consent

*Work in Progress*

*Severity*: Medium

*Guid*: 909aed8c-44cf-43b2-a381-8bafa2cf2149

Consent & Permissions: Allow group owner consent for selected group owners 

[Configure group owner consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent-groups?tabs=azure-portal)

In [ ]:
Test-GroupOwnerConsent -OutputToHost -ShowExplanation

##  Application Owners

*Severity*: High

*Guid*: N/A

MITRE ATT&CK tactics: [Persistence](https://attack.mitre.org/tactics/TA0003/), [Privilege Escalation](https://attack.mitre.org/tactics/TA0004/)

Credit: [Chad Cox](https://github.com/chadmcox) / [Applications/get-BuiltinAPPOwners.ps1](https://github.com/chadmcox/Azure_Active_Directory/blob/master/Applications/get-BuiltinAPPOwners.ps1)

Read here how these can be exploited: [Azure AD privilege escalation - Taking over default application permissions as Application Admin](https://dirkjanm.io/azure-ad-privilege-escalation-application-admin/) - Note that the Owner of the service principal can exploit this in the same way, hence why we look for owners.

Below code snippets look for various applications that are at an increased risk from having owners. 

In [ ]:
Find-OwnersFirstPartyMicrosoftApplications -OutputToHost -ShowExplanation

Look for applications with application permission in Microsoft Graph and 1 or more owners assigned. Application permissions are often medium-high risk permissions.

In [ ]:
Find-ApplicationsWithApplicationPermissionsAndOwner -OutputMarkdown -ShowExplanation

Look for applications with owners and any resource access that we do not consider low-risk. The applications listed below is worth looking into.

In [51]:
. "..\src\functions.ps1"
Show-LowRiskApplicationPermissions -ShowExplanation

These permissions are considered low risk

   PermissionType: Delegated

Id                                   Consent Name           Description
--                                   ------- ----           -----------
64a6cdd6-aab1-4aaf-94b8-3cc8405e90d0 User    email          Allows the app to read your primary em…
7427e0e9-2fba-42fe-b0c0-848c9e6a8182 User    offline_access Allows the app to see and update the d…
37f7f235-527c-4136-accd-4a02d197296e User    openid         Allows you to sign in to the app with …
14dad69e-099b-42c9-810b-d002981feec1 User    profile        Allows the app to see your basic profi…
e1fe6dd8-ba31-4d61-89e7-88639da4683d User    User.Read      Allows you to sign in to the app with …



In [53]:
Find-ApplicationsNonLowRiskPermissionsAndOwners -ShowExplanation -OutputMarkdown:$OutputMarkdown

Look for applications with owners and any resource access that we do not consider low-risk. 
Found no applications with Owners and non-low-risk permissions


## Applications with privileged app role assignments

All credit goes to [What's lurking in your Microsoft Graph app role assignments?](https://learningbydoing.cloud/blog/audit-ms-graph-app-role-assignments/)

In [ ]:
Get-PrivilegedAppRoleAssignments -OutputMarkdown:$OutputMarkdown

# Conditional Access Policies

## Block Legacy Protocols

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

[Common Conditional Access policy: Block legacy authentication](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-block-legacy)

This policy, along with policies that enforce MFA are the most important to have. Legacy authentication will de-facto bypass MFA.

Below looks for a conditional access policy that blocks legacy protocols and also outputs users excluded.

In [ ]:
Test-ConditionalAccessPolicy -BlockLegacyProtocols

## Require MFA for Administrators

*Severity*: High

*Guid*: fe1bd15d-d2f0-4d5e-972d-41e3611cc57b

[Common Conditional Access policy: Require MFA for administrators](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-admin-mfa)

Below looks for a conditional access policy that matches the policy template `"Require multifactor authentication for admins"`

In [ ]:
Test-ConditionalAccessPolicy -MfaAdministrators

## Require MFA for Azure Management

*Severity*: High

*Guid*: 4a4b1410-d439-4589-ac22-89b3d6b57cfc

[Common Conditional Access policy: Require MFA for Azure management](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-azure-management)

Below looks for a conditional access policy that matches the policy template `"Require multifactor authentication for Azure management"`

In [ ]:
Test-ConditionalAccessPolicy -MfaAzureManagement

## Restricted Locations

*Severity*: Medium

*Guid*: 079b588d-efc4-4972-ac3c-d21bf77036e5

[Using the location condition in a Conditional Access policy](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/location-condition)

Named locations can be used in numerous different way. A bad way to use them is to exclude from ex. enforcing MFA when coming from a `"trusted location"`. This does not conform to a `zero trust strategy`.

In [ ]:
# check if any named locations exist
$NamedLocation = Get-MgBetaIdentityConditionalAccessNamedLocation

$Compliant = $null -ne $NamedLocation -and $NamedLocation.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; $($NamedLocation.Count) Named Locations are defined" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No Named Locations are defined" -ForegroundColor Red
}

# we can look for policies that excludes locations under conditions but exclude those that block access
$Filter = "state eq 'enabled' and conditions/locations/excludeLocations/all(i:i ne null) and grantControls/builtInControls/all(i:i ne 'block')"
# need to use the beta API as v1.0 does not include policies made from templates
$PoliciesLocationExclusion = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

if($PoliciesLocationExclusion.Count -gt 0)
{
    Write-Warning "$($PoliciesLocationExclusion.Count) policies has location exclusions:"
    $Output = $PoliciesLocationExclusion | Select-Object -Property DisplayName, Id

    if($OutputMarkdown){$Output | ConvertTo-Markdown} else {$Output | Format-Table -AutoSize}
}

## Require devices to be marked as compliant

*Severity*: High

*Guid*: 7ae9eab4-0fd3-4290-998b-c178bdc5a06c

[Require device to be marked as compliant](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/concept-conditional-access-grant#require-device-to-be-marked-as-compliant)

Requiring devices to be marked as compliant in CA policy grants can be a powerful way of ensuring that connections are made from devices that are managed by the organization. With sufficiently strict device configurations enforced, this can be combined with MFA, or just a standalone grant.

In [ ]:
# we are looking for a policy that is enabled, the control is require device to be marked as compliant
$Filter = "state eq 'enabled' and grantControls/builtInControls/`$count gt 0 and grantControls/builtInControls/all(i:i eq 'compliantDevice')"
# need to use the beta API as v1.0 does not include policies made from templates
$CompliantDevicePolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $CompliantDevicePolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $CompliantDevicePolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $CompliantDevicePolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $CompliantDevicePolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

$Compliant = $null -ne $CompliantDevicePolicy -and $CompliantDevicePolicy.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found that requires devices to be marked as compliant:`n$($CompliantDevicePolicy.DisplayName -join ',')" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found" -ForegroundColor Red
}

if($null -ne $ExcludeUsers -and $ExcludeUsers.Count -gt 0){
    Write-Host "Excluded users:"
    $ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
        $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
        Write-Host "$($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
    }
}

## Protected Actions

Use [Protected Actions](https://learn.microsoft.com/en-us/azure/active-directory/roles/protected-actions-overview) to enforce strong authentcation and other strict grant controls when performing highly privileged actions, like `Delete conditional access policies`.

In [ ]:
# look for protected actions with an authentication context
# Get-MgBetaRoleManagementDirectoryResourceNamespaceResourceAction seems broken. what is UnifiedRbacResourceNamespaceId
$Response = Invoke-MgGraphRequest -Method GET "https://graph.microsoft.com/beta/roleManagement/directory/resourceNamespaces/microsoft.directory/resourceActions?`$filter=isAuthenticationContextSettable eq true and authenticationContextId ne null&`$select=id,name,description,actionVerb,isEnabledForCustomRole,isAuthenticationContextSettable,authenticationContextId&`$top=500" -ErrorAction Stop
# first we check if any protected actions are defined at all
$ProtectedActions = $Response.value
if($ProtectedActions.Count -eq 0)
{
    Write-Warning "No protected actions defined"
    return
}
# get all possible protected actions
$Response = Invoke-MgGraphRequest -Method GET "https://graph.microsoft.com/beta/roleManagement/directory/resourceNamespaces/microsoft.directory/resourceActions?`$filter=isAuthenticationContextSettable eq true&`$select=id" -ErrorAction Stop
Write-Host "$($ProtectedActions.Count) of $($Response.value.Count) protected actions are associated with an authentication context"

$authenticationContextIds = ($ProtectedActions | Select-Object -ExpandProperty authenticationContextId) | Sort-Object -Unique
# next we find if these are in use in any CA Policy
$FilterArray = ($authenticationContextIds | ForEach-Object { "'$_'"}) -join ','
$Filter = "state eq 'enabled' and grantControls/builtInControls/`$count gt 0 and conditions/applications/includeAuthenticationContextClassReferences/`$count gt 0 and conditions/applications/includeAuthenticationContextClassReferences/all(i:i in ($FilterArray))" 
# need to use the beta API as v1.0 does not include policies made from templates
$CompliantDevicePolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter
if($CompliantDevicePolicy.Count -eq 0)
{
    Write-Warning "No CA policy uses the authentication context associated with a protected action"
}
else
{
    # TODO: specify which action is protected by which policy. Most likely that they are all protected by a single policy, so not that important.
    Write-Host "Protected actions: $(($ProtectedActions | Select-Object -ExpandProperty description) -join ',') are protected by CA policies: $(($CompliantDevicePolicy | Select-Object -ExpandProperty DisplayName) -join ',')"
}

## Break Glass/Emergency Accounts

*Work in Progress*

Count how many times all user objects are excluded from CA policies. If there is one with a high count this could be the emergency account(s)

# Password Reset

## Self-service password reset policy requirement

*Work in Progress*

*Severity*: High

*Guid*: 4c1e945b-459c-4373-b7ed-71623b375a91

## Re-confirm authentication information

*Work in Progress*

*Severity*: Medium

*Guid*: 7ecbe48f-be64-4dd7-bf2e-8bbbc468155a

## Number of methods required to reset password

*Work in Progress*

*Severity*: High

*Guid*: bc9164e9-909a-4ed8-a44c-f3b2b3818baf